In [6]:
# !pip install openai

In [20]:
from openai import OpenAI
import base64


In [8]:
client = OpenAI()


In [18]:
SYSTEM_PROMPT = """
Talk to the user as if they were a pirate.
"""

#### The Basics


In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": "Write a haiku about the color red."
        }
    ]
)

print(completion.choices[0].message)
completion.choices[0].message.content


#### Function Calling

In [ ]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "get_weather",
          "parameters": {
              "type": "object",
              "properties": {
                  "location": {"type": "string"}
              },
          },
      },
  }
]

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role": "user", "content": "What's the weather like in Paris today?"}],
  tools=tools,
)

print(completion.choices[0].message.tool_calls)
completion.choices[0].message.content


#### With Vision 👀

In [ ]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "../assets/cole-mader-9WV1S3JsWm8-unsplash.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
)

# print(response.choices[0])
response.choices[0].message.content
